In [174]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [175]:
### Load the ANN trained model, scaler pickle file, onehot encoding

model = tf.keras.models.load_model(r"E:\Python - Intellipaat_D\Gen AI\Simple ANN\model.h5")

with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file)

with open("one_hot_encoder_gro.pkl", "rb") as file:
    ohe_encoder_geo = pickle.load(file)

with open("standard_scaler.pkl", "rb") as file:
    stand_scaler = pickle.load(file)

In [176]:
## Example input data
input_data = {
    'CreditScore': 0,
    'Geography': "France",
    'Gender': "Male", 
    'Age': 72, 
    'Tenure': 0, 
    'Balance': 0, 
    'NumOfProducts': 0, 
    'HasCrCard': 0,
    'IsActiveMember':0, 
    'EstimatedSalary': 0
}
input_data

{'CreditScore': 0,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 72,
 'Tenure': 0,
 'Balance': 0,
 'NumOfProducts': 0,
 'HasCrCard': 0,
 'IsActiveMember': 0,
 'EstimatedSalary': 0}

In [177]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0,France,Male,72,0,0,0,0,0,0


In [178]:
## Endde the Categorical Variable
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,0,France,1,72,0,0,0,0,0,0


In [179]:
geo_encoder = ohe_encoder_geo.transform(input_df[["Geography"]])
geo_encoder

<1x3 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [180]:
geo_encoder.toarray(), ohe_encoder_geo.get_feature_names_out(["Geography"])

(array([[1., 0., 0.]]),
 array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
       dtype=object))

In [181]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns =ohe_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [182]:
input_df = pd.concat([input_df.drop("Geography", axis = 1), geo_encoder_df], axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,0,1,72,0,0,0,0,0,0,1.0,0.0,0.0


In [183]:
input_df.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [184]:
## Scaling the Data
input_df_scaled = stand_scaler.transform(input_df)
input_df_scaled

array([[-6.76262379,  0.91324755,  3.14711155, -1.73646664, -1.21847056,
        -2.6418115 , -1.54035103, -1.02583358, -1.74616572,  1.00150113,
        -0.57946723, -0.57638802]])

In [185]:
### Prediction
prediction = model.predict(input_df_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


array([[0.8618547]], dtype=float32)

In [186]:
prediction_prob = prediction[0][0]
prediction_prob

0.8618547

In [187]:
if prediction_prob > 0.5:
    print("The customer is likely to Churn")
else:
    print("The customer is not likely to churn")

The customer is likely to Churn
